# Load and pre-process data

## Investigate dataset folder

the dog breed dataset folder includes three main folders: train, valid and test. As suggested by its name, images in folder are the direct input where the learning step bases on to estimate gradient and update the parameters. Images in the validation folder is for evaluating the accurary of the model after each epoch. The last folder, test, contains images which we will use at the last step to calculate the model's accuracy. These images are considered as the real world data, which our model has never seen, and is reliable for evaluation purpose.

Because each dog breed is stored in a separate folder, we use the folder name as the label for the underlying images. 

In [3]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset(f'dogImages/train')
valid_files, valid_targets = load_dataset(f'dogImages/valid')
test_files, test_targets = load_dataset(f'dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob(f'dogImages/train/*/'))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


## Prepare input tensors

In both training and testing step, the input to neural network are $4D$ tensor as following:

$$
(\text{n_samples},  \text{rows}, \text{columns}, \text{channels})
$$

where `n_samples` is the total number of images, `rows` and `columns` are size and `channels` are the number of channels of each image. In our case, `channels` is 3, equivalent to  `RGB` channels.

The function `path_to_tensor` takes input as a path to a single image, import, resize to the size of $(224,224)$ and then convert it to a 4D tensor, whose shape is
$$
(\text{1},  \text{rows}, \text{columns}, \text{channels})
$$

The function `paths_to_tensor` receives input as a list of path to images, forward each path to the function `path_to_sensor` to get a single 4D tensor, and finally, stack all of them together to build the final tensor.  

We repeat calling `paths_to_tensor` to build tensors for training, valid and test sets.

After loading all tensors, we divide images by 255 to convert pixel values to the range of $[0, 1]$

In [4]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [5]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:07<00:00, 116.47it/s]


# a try to train a model from scratch